In [1]:
from IPython.display import *
!git clone https://github.com/MCAIGC/tts_metrics
%cd tts_metrics
# 下载模型
!mkdir models
!wget https://huggingface.co/Dongchao/UniAudio/resolve/main/wavlm_large_finetune.pth -O ./models/wavlm_large_finetune.pth
clear_output() 

# 导入必需库

In [2]:
from IPython.display import *
import gc
import ctypes
import torch
# Function to clean RAM & vRAM
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()
clear_output()
import math
import os
import random
import string
from pathlib import Path

import torch
import torch.nn.functional as F
import torchaudio
from tqdm import tqdm
import warnings 
warnings.filterwarnings('ignore',category=UserWarning)
warnings.filterwarnings('ignore',category=FutureWarning)
!pip list|grep torch
!pip list|grep transformers

pytorch-ignite                     0.5.1
pytorch-lightning                  2.5.0.post0
torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchinfo                          1.8.0
torchmetrics                       1.6.1
torchsummary                       1.5.1
torchtune                          0.5.0
torchvision                        0.20.1+cu121
sentence-transformers              3.3.1
transformers                       4.47.0


# 加载wavlm_large模型

In [3]:
from ecapa_tdnn import ECAPA_TDNN_SMALL
use_gpu = True
clear_output()
wavlm_ckpt_dir = "./models/wavlm_large_finetune.pth"
model = ECAPA_TDNN_SMALL(feat_dim=1024, feat_type="wavlm_large", config_path=None)
state_dict = torch.load(wavlm_ckpt_dir, weights_only=True, map_location=lambda storage, loc: storage)
model.load_state_dict(state_dict["model"], strict=False)
use_gpu = True if torch.cuda.is_available() else False
if use_gpu:
    model = model.cuda()
model.eval()
print("模型加载成功")

Downloading: "https://github.com/s3prl/s3prl/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: https://huggingface.co/s3prl/converted_ckpts/resolve/main/wavlm_large.pt
Destination: /root/.cache/s3prl/download/f2d5200177fd6a33b278b7b76b454f25cd8ee866d55c122e69fccf6c7467d37d.wavlm_large.pt
100%|██████████| 1.18G/1.18G [00:05<00:00, 228MB/s]


模型加载成功


# 判断说话人声音是否相似

In [4]:
def run_sim(wav1_path,wav2_path ):
    wav1, sr1 = torchaudio.load(wav1_path)
    wav2, sr2 = torchaudio.load(wav2_path)
    
    resample1 = torchaudio.transforms.Resample(orig_freq=sr1, new_freq=16000)
    resample2 = torchaudio.transforms.Resample(orig_freq=sr2, new_freq=16000)
    wav1 = resample1(wav1)
    wav2 = resample2(wav2)
    
    if use_gpu:
        wav1 = wav1.cuda()
        wav2 = wav2.cuda()
    with torch.no_grad():
        emb1 = model(wav1)
        emb2 = model(wav2)
    
    sim = F.cosine_similarity(emb1, emb2)[0].item()
    return sim

In [7]:
%%time 
wav1_path = "ZH_2_prompt.wav"
wav2_path = "ZH_2_1.wav"
run_sim(wav1_path,wav2_path )

CPU times: user 628 ms, sys: 250 ms, total: 878 ms
Wall time: 1.41 s


0.7926095724105835